In [1]:
import torch
import numpy as np
import torch.nn as nn
import math
import os
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F



In [2]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        #does not pass all input words with eachother. Each word goes through independantly
        #and the output are the embeddings of the word. We want this because we do not 
        #want to concacenate the embeddings to the output nodes.
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) 
        #now takes in all embeddings of each word stretched out
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        self.tokens = {}

    def forward(self, inputs):
        #embeds will be flattened matrix
        embeds = self.embeddings(inputs).view((1, -1))
        #rectified relu to learn embeddings
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        #output is the log probablities of all vocabulary
        return log_probs

    


In [19]:
#file path to credit card csv file
file_path = os.path.join(os.path.expanduser("~"), "Downloads", "mental_health.csv")
orig_dataset = pd.read_csv(file_path) #read csv file as pandas object
orig_dataset = orig_dataset.to_numpy()
print("Dataset size: "+ str(len(orig_dataset)))
dataset = orig_dataset[0:100] #use part of the dataset
print("Train Dataset size: "+ str(len(dataset)))
minFreq = {} #word must appear n times to be added to dictionary
dictionary = {} #relevant words in the dicationary
index = 2
for example in range(len(dataset)):
    for cont_response in range(2):
        if type(dataset[example][cont_response]) == float: #NaN values
            continue
        for word in dataset[example][cont_response].split():
            if word not in minFreq:
                minFreq[word]=1
            else:
                if minFreq[word]==3: #word needs to appear
                    dictionary[word] = index
                    index+=1
                minFreq[word]+=1

print( "Total Dictionary Size: 10,489")
print("Training Dictionary Size: " + str(index))

CONTEXT_SIZE = 3 #look 3 words back to predict current word
EMBEDDING_DIM = 252 #total embeddings for each word
all_ngrams = [] #ngram setup -> [(['through', 'going', "I'm"], 'some')]
for example in range(len(dataset)): 
    for cont_response in range(2): #context than response
        if type(dataset[example][cont_response]) == float: #NaN values
            continue
        cur_Sentence = dataset[example][cont_response].split() #seperate by word
        ngrams = [ #[(['through', 'going', "I'm"], 'some')]
            ([cur_Sentence[i - j - 1] for j in range(CONTEXT_SIZE)],cur_Sentence[i])
            for i in range(CONTEXT_SIZE, len(cur_Sentence))
            ]
        #append the grams to all_ngrams
        for i in ngrams:
            all_ngrams.append(i) 
loss_function = nn.NLLLoss() #loss layer
model = NGramLanguageModeler(index, EMBEDDING_DIM, CONTEXT_SIZE) #intialize Ngram model
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.tokens = dictionary
for epoch in range(10):
    total_loss = 0
    print("Epoch: "+ str(epoch))
    maxFreq = 3 #max number of times a word can be trained
    #dictionary to keep track of times word is trained. Will skip if words have been trained maxFreq times
    maxFreqDict = {}
    for context, target in all_ngrams:
        #if unknown word, just don't train
        if context[0] not in dictionary or context[1] not in dictionary or context[2] not in dictionary:
                continue
        if target not in dictionary:
                continue
        #add context words if not found in dict
        if context[0] not in maxFreqDict:
            maxFreqDict[context[0]] = 1
        if context[1] not in maxFreqDict:
            maxFreqDict[context[1]] = 1
        #if both words have been trained equal to or more than maxFreq times, continue
        #already has been trained enough
        if maxFreqDict[context[0]] >= maxFreq and maxFreqDict[context[1]] >= maxFreq:
            continue
        #update how many times the context words have been trained
        maxFreqDict[context[0]]+=1
        maxFreqDict[context[1]]+=1
            
        #turn each word to an integer and wrapped in tensor so pass as an input to the model
        context_idxs = torch.tensor([dictionary[w] for w in context], dtype=torch.long)

        #zero out gradients cause it accumulates
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        
        
            

        #apply the loss function to the log probabilties with the correct target word
        loss = loss_function(log_probs, torch.tensor([dictionary[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(f"Total_Loss: {total_loss}")





Dataset size: 3512
Train Dataset size: 100
Total Dictionary Size: 10,489
Training Dictionary Size: 586
Epoch: 0
Total_Loss: 5902.9905116558075
Epoch: 1
Total_Loss: 4411.793348670006
Epoch: 2
Total_Loss: 3575.1542907953262
Epoch: 3
Total_Loss: 3001.015433639288
Epoch: 4
Total_Loss: 2472.4445091485977
Epoch: 5
Total_Loss: 1945.913971543312
Epoch: 6
Total_Loss: 1431.9736630246043
Epoch: 7
Total_Loss: 975.9797193184495
Epoch: 8
Total_Loss: 625.2487171627581
Epoch: 9
Total_Loss: 387.28769674524665


In [20]:
torch.save(model.state_dict(), "embedding_model")

In [21]:
print(model.embeddings(torch.tensor([0])))

tensor([[-0.2561,  0.3934,  0.5447,  1.0223, -0.3705,  1.0004, -1.2525,  0.9334,
          1.1842, -0.3008, -0.2600,  1.6086, -0.3246, -0.9873, -0.2432, -0.2886,
          0.1233,  2.4316,  0.2166, -1.6824,  0.0320, -1.1318,  0.1125,  1.0177,
          0.3158,  1.3555,  1.5110,  0.0643, -1.0691,  0.1664, -0.4808,  0.4448,
          0.1906, -0.2950,  1.2843, -0.4583,  0.7103,  0.5837, -0.7791,  0.6432,
          0.2725,  0.1528, -0.4040,  0.8149,  1.6782, -0.2662,  0.1618,  1.4832,
         -1.7338, -0.4428, -0.1752,  1.1027, -0.1932,  1.2469, -0.5988,  0.8388,
          0.6997, -0.3709,  0.5973,  0.2280, -0.2573,  1.7032,  1.1642,  0.1414,
          1.5644,  1.6152, -0.5278,  1.9608, -1.3675,  0.1461, -2.5155, -1.2850,
         -0.3326, -0.8356, -0.8628,  0.9947,  0.2619,  0.2297, -1.2591,  0.3810,
         -0.7353,  1.5858,  0.4280, -1.0719, -0.7522,  0.9927, -0.0633, -1.7193,
          1.5215,  0.3605,  0.5542,  0.4022,  1.4687,  0.2079,  0.2285, -0.7784,
         -0.0824, -0.2949,  